In [14]:
import numpy as np

"""
ステアリングベクトルを算出
mic_alignments: 3xM ndarray [[x, y, z], [x, y, z]] (書籍内でのd_mやd_nに相当)
source_locations: 3xN_s ndarray [[x, y, z], [x, y, z]] （書籍内でのpに相当）
    恐らくだが、[[x_1, x_2], [y_1, y_2], [z_1, z_2]] の方が正しい
freqs: 1xN_k array
sound_speed: 音速 [m/s]
is_use_far: Farを使う場合はTrue, Nearの場合はFalse
return: steering vector(N_k x N_s x M)
"""
# このx, y, z はどの方向の座標を表している？


def calculate_steering_vector(mic_alignments, source_locations, freqs, sound_speed=340, is_use_far=True):
    n_channels = np.shape(mic_alignments)

    n_sources = np.shape(source_locations)

    if is_use_far == True:
        norm_source_locations = source_locations/np.linalg.norm(source_locations, 2, axis=0, keepdims=True)
        # numpyの機能で自動で計算する次元を合わせてくれる？らしい
        # アインシュタインの縮約記法を理解する必要あり
        # あらかた理解したが下の計算は何をやってるのか理解できない、、、

        print("norm_source_locations:\n", norm_source_locations[..., None], "\n")
        print("mic_alignments:\n", mic_alignments[:, None, :])

        steering_phase = np.einsum('k,ism,ism->ksm', 2.j*np.pi/sound_speed*freqs, norm_source_locations[..., None], mic_alignments[:, None, :])
        steering_vector = 1./np.sqrt(n_channels)*np.exp(steering_phase)

        return steering_vector

        # steering_phase_2 = np.einsum('k,ism,ims->ksm', 2.j*np.pi/sound_speed*freqs, norm_source_locations[..., None], mic_alignments[:, None, :])
        # steering_vector_2 =1./np.sqrt(n_channels)*np.exp(steering_phase_2)

        # return([steering_vector, steering_phase_2])



In [15]:
sample_rate = 16000

N = 1024

Nk = N/2 + 1

# sample_rate/Nが周波数インデックス1つ分になるっぽい
freqs = np.arange(0, Nk, 1) * sample_rate / N

mic_alignments = np.array(
    [
        [-0.01, 0.0, 0.0],
        [0.01, 0.0, 0.0],
    ]
).T

doas = np.array(
    [[np.pi/2, 0],
     [np.pi/2, np.pi]
    ])

distance = 1

source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
source_locations[0, :] = np.cos(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[1, :] = np.sin(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[2, :] = np.cos(doas[:, 0])

source_locations *= distance

far_steering_vectors=calculate_steering_vector(mic_alignments=mic_alignments, source_locations=source_locations, freqs=freqs)

# 周波数インデックス x 音源番号 x マイク番号
# Nk x Ns x M
print(far_steering_vectors.shape)

# print(far_steering_vectors)
# print(np.array_equal(far_steering_vectors[0],far_steering_vectors[1]))

norm_source_locations:
 [[[ 1.0000000e+00]
  [-1.0000000e+00]]

 [[ 0.0000000e+00]
  [ 1.2246468e-16]]

 [[ 6.1232340e-17]
  [ 6.1232340e-17]]] 

mic_alignments:
 [[[-0.01  0.01]]

 [[ 0.    0.  ]]

 [[ 0.    0.  ]]]
(513, 2, 2)


In [2]:
# einsumの実験用コード

import numpy as np

a = np.array([[0, 1, 2], [3, 4, 5]]).T
print(a)
print(a[:, 0])
print(a[0, :])

print(np.linalg.norm(a, 2, axis=0, keepdims=False))
print(np.linalg.norm(a, 2, axis=0, keepdims=True))
print(np.linalg.norm(a, 2, axis=1, keepdims=False))
print(np.linalg.norm(a, 2, axis=1, keepdims=True))

# aaa = a/np.linalg.norm(a, 2, axis=0, keepdims=True)
# print("..., None: \n", aaa[..., None])

b = np.array([[0, 1], [2, 3], [4, 5]])
bbb = b/np.linalg.norm(b, 2, axis=0, keepdims=True)
print("..., None: \n", bbb[..., None])

print(":, None, N: \n", a[:, None, :])


# やっぱり下の奴のほうがテンソルの形は同じになる
# print(":, None, N: \n", a[..., None])


# print("b:")
# b = np.array([[0, 1, 6], [2, 3, 7], [4, 5, 8]])
# print(np.linalg.norm(b, 2, axis=0, keepdims=True))
# print("keepdims=True: \n", b/np.linalg.norm(b, 2, axis=0, keepdims=True), "\n")
# print("keepdims=False: \n" , b/np.linalg.norm(b, 2, axis=0, keepdims=False), "\n")

doas = np.array([[np.pi/2,0], [np.pi/2, np.pi]])

source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
print(source_locations)
source_locations[0, :] = np.cos(doas[:, 1]) * np.sin(doas[:, 0])
print(source_locations)
source_locations[1, :] = np.sin(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[2, :] = np.cos(doas[:, 0])
print(source_locations)



[[0 3]
 [1 4]
 [2 5]]
[0 1 2]
[0 3]
[2.23606798 7.07106781]
[[2.23606798 7.07106781]]
[3.         4.12310563 5.38516481]
[[3.        ]
 [4.12310563]
 [5.38516481]]
..., None: 
 [[[0.        ]
  [0.16903085]]

 [[0.4472136 ]
  [0.50709255]]

 [[0.89442719]
  [0.84515425]]]
:, None, N: 
 [[[0 3]]

 [[1 4]]

 [[2 5]]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[ 1. -1.]
 [ 0.  0.]
 [ 0.  0.]]
[[ 1.0000000e+00 -1.0000000e+00]
 [ 0.0000000e+00  1.2246468e-16]
 [ 6.1232340e-17  6.1232340e-17]]
ベクトルの積の実験
[3 8]
[ 5 12]


In [3]:
print("ベクトルの積の実験")
a = np.array([1, 2])
b = np.array([3, 4]).T
c = np.array([5, 6])

# どうやら普通の乗算記号では対応する要素ごとの掛け算がされるだけらしい
print(a * b)
print(a * c)

ベクトルの積の実験
[3 8]
[ 5 12]
